In [1]:
from typing import Any , Dict
from google.adk.agents import Agent , LlmAgent
from google.adk.apps.app import App , EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService, InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import ToolContext
from google.genai import types

In [3]:
#Agent Executor Function

async def agent_invoke(
        agent: Runner,
        user_input: list[str] | str = None,
        session_name :str = "default_session",
):
    
   
    app_name = agent.app_name

    session = await session_service.get_session(app_name = app_name, session_id = session_name, user_id = USER_ID)

    if session:
        print(f"Resuming session: {session_name}")
    else:
        session = await session_service.create_session(app_name = app_name, session_id = session_name, user_id = USER_ID)
        print(f"creating new session: {session_name}")

    if user_input:
        if type(user_input) == str:
            user_input = [user_input]

        for msg in user_input:
            print(f"Human: {msg}")

            msg = types.Content(role="Human", parts=[types.Part(text = msg)])    

            async for event in runner_instance.run_async(user_id = USER_ID , session_id = session.id , new_message = msg):
                if event.content and event.content.parts:
                    print(f"Agent: {event.content.parts[0].text}")
    else:
        print(f"No querries found.")